In [1]:
import time
import numpy as np
from scipy.special import erfc
import mfem.ser as mfem
from mfem.ser import intArray
from glvis import glvis

In [2]:
order = 3
vis_steps = 5
sleep_interval = 0.05

mesh = "periodic-hexagon.mesh"
ref_levels = 2
dt = 0.01
t_final = 10

ode_solver = mfem.RK4Solver()



In [3]:
mesh_url = f"https://raw.githubusercontent.com/mfem/mfem/master/data/{mesh}"
mesh_str = !curl -s {mesh_url}
mesh_file = f"./data/{mesh}"
with open(mesh_file, "w") as f:
    f.write("\n".join(mesh_str))
mesh = mfem.Mesh(mesh_file, 1,1)
dim = mesh.Dimension()

In [4]:
for lev in range(ref_levels):
    mesh.UniformRefinement()
    if mesh.NURBSext:
        mesh.SetCurvature(max(order, 1))
    bb_min, bb_max = mesh.GetBoundingBox(max(order, 1))

In [5]:
fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)

print("Number of unknowns: " + str(fes.GetVSize()))

class velocity_coeff(mfem.VectorPyCoefficient):
    def EvalValue(self, x):        
        dim = len(x)
        
        center = (bb_min + bb_max)/2.0
        # map to the reference [-1,1] domain                
        X = 2 * (x - center) / (bb_max - bb_min)
        if dim == 1: v = [1.0,]
        elif dim == 2: v = [np.sqrt(2./3.), np.sqrt(1./3)]
        elif dim == 3: v = [np.sqrt(3./6.), np.sqrt(2./6), np.sqrt(1./6.)]
        
        return v
      
        
class u0_coeff(mfem.PyCoefficient):
    def EvalValue(self, x):        
        dim = len(x)
        center = (bb_min + bb_max)/2.0
        # map to the reference [-1,1] domain        
        X = 2 * (x - center) / (bb_max - bb_min)
        if dim == 1:
            return np.exp(-40. * (X[0]-0.5)**2)
        elif (dim == 2 or dim == 3):
            rx = 0.45 
            ry = 0.25
            cx = 0.
            cy = -0.2
            w = 10.
            if dim == 3:
                s = (1. + 0.25*np.cos(2 * np.pi * x[2]))
                rx = rx * s
                ry = ry * s
            return ( erfc( w * (X[0]-cx-rx)) * erfc(-w*(X[0]-cx+rx)) *
                        erfc( w * (X[1]-cy-ry)) * erfc(-w*(X[1]-cy+ry)) )/16
        
       
        return 0.0

class inflow_coeff(mfem.PyCoefficient):
    def EvalValue(self, x):
        return 0

Number of unknowns: 3072


In [6]:
velocity = velocity_coeff(dim)
inflow = inflow_coeff()
u0 = u0_coeff()

m = mfem.BilinearForm(fes)
m.AddDomainIntegrator(mfem.MassIntegrator())
k = mfem.BilinearForm(fes)
k.AddDomainIntegrator(mfem.ConvectionIntegrator(velocity, -1.0))
k.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(velocity, 1.0, -0.5)))
k.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(velocity, 1.0, -0.5)))
b = mfem.LinearForm(fes)
b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow, velocity, -1.0, -0.5))

m.Assemble()
m.Finalize()
skip_zeros = 0
k.Assemble(skip_zeros)
k.Finalize(skip_zeros)
b.Assemble()

# Initial conditions
u = mfem.GridFunction(fes)
u.ProjectCoefficient(u0)

In [7]:
g = glvis((mesh, u))
g.render()

class FE_Evolution(mfem.PyTimeDependentOperator):
    def __init__(self, M, K, b):
        mfem.PyTimeDependentOperator.__init__(self, M.Size())
        self.K = K        
        self.M = M
        self.b = b
        self.z = mfem.Vector(M.Size())
        self.zp = np.zeros(M.Size())
        self.M_prec = mfem.DSmoother()        
        self.M_solver = mfem.CGSolver()
        self.M_solver.SetPreconditioner(self.M_prec)        
        self.M_solver.SetOperator(M)
        self.M_solver.iterative_mode = False
        self.M_solver.SetRelTol(1e-9)
        self.M_solver.SetAbsTol(0.0)
        self.M_solver.SetMaxIter(100)
        self.M_solver.SetPrintLevel(0)
        
    def Mult(self, x, y):
        self.K.Mult(x, self.z)
        self.z += b
        self.M_solver.Mult(self.z, y)

adv = FE_Evolution(m.SpMat(), k.SpMat(), b)

ode_solver.Init(adv)
t = 0.0
ti = 0
time.sleep(1)
while True:
    if t > t_final - dt/2: break
    t, dt = ode_solver.Step(u, t, dt)
    ti = ti + 1
    if ti % vis_steps == 0:
        g.update((mesh, u))
        time.sleep(sleep_interval)
        print(f"time step: {ti}, time: {t:.2e}", end="\r")

glvis(data_str='MFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see mesh/geom.hpp):\n#\n# POINT       = 0\n# SEGME…